# firm-firm-year 로 변경

### 1. citation
- pat_cit, app_cit edge_list 사용
- gvkey / gvkey_cited / year / pat_cit (app_cit) 형태로 만들기
- outer join

### 2. litigation
- lit_temp 사용
- defend: caseid / gvkey / year / counter, attack: caseid / gvkey_cited
- caseid 기준 left join
- groupby -> lit_t1, lit_t1_c

### 3. tech proximity
- 현재 산업별, 연도별 따로 존재
- 각 파일 edgelist로 만들기
- 산업별로 뭉치기

### 4. join
- lit_t1 + lit_c: left, fillna(0)
- '+' cit: outer, fillna(0)
- '+' proxy: left

### 1. citation

In [1]:
import pandas as pd
import numpy as np
import csv
import os
import math

os.chdir('E:/apps')

In [3]:
pb = pd.read_csv('pat_cit_edge_BIO.csv')
pe = pd.read_csv('pat_cit_edge_EE&IT.csv')

pb = pb.groupby(['gvkey','gvkey_cited','year']).size().reset_index(name='pat_cit')
pe = pe.groupby(['gvkey','gvkey_cited','year']).size().reset_index(name='pat_cit')

pb.iloc[0:10,:]

,gvkey,gvkey_cited,year,pat_cit
0,1078,1078.0,2005,490
1,1078,1078.0,2006,968
2,1078,1078.0,2007,1151
3,1078,1078.0,2008,1050
4,1078,1078.0,2009,1487
5,1078,1078.0,2010,2616
6,1078,1078.0,2011,3647
7,1078,1078.0,2012,3826
8,1078,1602.0,2007,4
9,1078,1602.0,2008,1


In [8]:
len(pb), len(pe)

(23957, 130617)

In [10]:
pb = pb.loc[pb['gvkey']!=pb['gvkey_cited'],:].reset_index(drop=True)
pe = pe.loc[pe['gvkey']!=pe['gvkey_cited'],:].reset_index(drop=True)

len(pb), len(pe)

(22404, 128099)

In [15]:
ab = pd.read_csv('app_cit_edge_bio.csv')
ae = pd.read_csv('app_cit_edge_EE&IT.csv')

ab = ab.groupby(['gvkey','gvkey_cited','year']).size().reset_index(name='app_cit')
ae = ae.groupby(['gvkey','gvkey_cited','year']).size().reset_index(name='app_cit')

len(ab), len(ae)

(5134, 32650)

In [16]:
ab = ab.loc[ab['gvkey']!=ab['gvkey_cited'],:].reset_index(drop=True)
ae = ae.loc[ae['gvkey']!=ae['gvkey_cited'],:].reset_index(drop=True)

len(ab), len(ae)

(4148, 30966)

In [17]:
ab.iloc[0:10,:]

,gvkey,gvkey_cited,year,app_cit
0,1078,1602.0,2010,1
1,1078,1602.0,2011,2
2,1078,1602.0,2012,1
3,1078,2044.0,2007,4
4,1078,2044.0,2008,3
5,1078,2044.0,2009,4
6,1078,2044.0,2010,6
7,1078,2044.0,2011,12
8,1078,2044.0,2012,8
9,1078,2085.0,2008,1


In [18]:
pb = pb.merge(ab,how='outer',on=['gvkey','gvkey_cited','year']).fillna(0)
pe = pe.merge(ae,how='outer',on=['gvkey','gvkey_cited','year']).fillna(0)

len(pb), len(pe)

(23418, 134016)

In [19]:
pb.to_csv('feature_ffy/cit_bio.csv',index=False)
pe.to_csv('feature_ffy/cit_EE&IT.csv',index=False)

### 2. litigation

In [2]:
os.chdir('E:/apps/db_table')

df = pd.read_csv('lit_temp.csv')
del df['date_filed']

attack = ['Counter Claimant','Plaintiff','Counter-claimant','Consol Counter Claimant','Consol Plaintiff','ThirdParty Plaintiff',
          'Counter Plaintiff','Counter-Claimant','Third Party Plaintiff','3rd party plaintiff','Counterclaim Plaintiff']
defend = ['Defendant','Counter Defendant','Consol Defendant','Counter-defendant','ThirdParty Defendant','Consol Counter Defendant',
         'Third Party Defendant','3rd party defendant','Counter-Defendant','Consolidated Defendant','Counterclaim Defendant']

atk = df.loc[df['party_type'].isin(attack),:]
dfd = df.loc[df['party_type'].isin(defend),:]

len(atk), len(dfd)

(28494, 35720)

In [3]:
bio = pd.read_csv('litigation_gvkey_BIO.csv')
ee = pd.read_csv('litigation_gvkey_EE&IT.csv')

bioa = bio.merge(atk,how='left',on='name')
biod = bio.merge(dfd,how='left',on='name')

eea = ee.merge(atk,how='left',on='name')
eed = ee.merge(dfd,how='left',on='name')

In [4]:
l = [bioa,biod,eea,eed]

for df in l:
    del df['name']
    df.dropna(how='any',inplace=True)
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)

In [5]:
I = [bioa,eea]

for df in I:
    del df['party_type']
    df.rename(columns={"gvkey": "gvkey_cited"},inplace=True)

list(bioa)

['gvkey_cited', 'case_row_id', 'year']

In [6]:
dcnt = ['Counter Defendant','Counter-defendant','Consol Counter Defendant','Counter-Defendant','Counterclaim Defendant']

def counter_d(party):
    dum=1 if party in dcnt else 0
    return dum

d = [biod,eed]
        
for df in d:
    df['counter'] = df['party_type'].apply(counter_d,1)
    del df['party_type']

In [7]:
biod.iloc[0:10,:]

,gvkey,case_row_id,year,counter
0,1078,38104.0,2009.0,0
1,1078,54941.0,2009.0,0
2,1078,16362.0,2009.0,1
3,1078,17361.0,2010.0,0
4,1078,15824.0,2009.0,1
5,1078,16490.0,2010.0,1
6,1078,17105.0,2010.0,0
7,1078,18303.0,2011.0,1
8,1078,19092.0,2012.0,1
9,1078,35734.0,2006.0,0


In [8]:
bio = biod.merge(bioa,how='left',on=['case_row_id','year'])
ee = eed.merge(eea,how='left',on=['case_row_id','year'])

len(bio),len(ee)

(654, 5589)

In [9]:
bio = bio[['gvkey','gvkey_cited','year','counter','case_row_id']]
bio.dropna(how='any',inplace=True)
bio = bio.loc[bio['gvkey']!=bio['gvkey_cited'],:].reset_index(drop=True)
len(bio)

281

In [10]:
ee = ee[['gvkey','gvkey_cited','year','counter','case_row_id']]
ee.dropna(how='any',inplace=True)
ee = ee.loc[ee['gvkey']!=ee['gvkey_cited'],:].reset_index(drop=True)
len(ee)

3676

In [11]:
def get_lit_var(df):    
    df1 = df.loc[:,['gvkey','gvkey_cited','year']]
    df1['year'] = df1['year']-1
    df1['lit_t1'] = 0
    df1 = df1.groupby(['gvkey','gvkey_cited','year'], as_index=False).count()    
    
    df2 = df.loc[:,['gvkey','gvkey_cited','year','counter']]
    df2 = df2.loc[df2['counter']==1,['gvkey','gvkey_cited','year']]
    df2['year'] = df2['year']-1
    df2['lit_t_c'] = 0
    df2 = df2.groupby(['gvkey','gvkey_cited','year'], as_index=False).count()
    
    return [df1,df2]

In [18]:
a = get_lit_var(bio)
b = get_lit_var(ee)
c = ['t1','t1_c']

for i in range(0,2):
    a[i].to_csv('E:/apps/feature_ffy/lit_'+c[i]+'_bio.csv',index=False)
    b[i].to_csv('E:/apps/feature_ffy/lit_'+c[i]+'_EE&IT.csv',index=False)

### 3. tech proxy

In [1]:
import pandas as pd
import numpy as np
import csv
import os
import math

os.chdir('E:/apps/patcit measure')

In [4]:
def get_edge(ind):
    fname = 'E:/apps/feature_ffy/proximity_' + ind + '.csv'
    
    with open(fname,'wb') as w:
        writer = csv.writer(w)
        writer.writerow(['gvkey','gvkey_cited','year','t_proxy'])
        
        for y in range(2006,2013):
            fread = 'proximity_' + ind + '_' + str(y) +'.csv'
            with open(fread,'rb') as f:
                reader = csv.reader(f)
                header = reader.next()[1:]
                i=1
                for line in reader:
                    gvkey = line[0]
                    year = str(y)
                    for key, value in zip(header[i:],line[i+1:]):
                        writer.writerow([gvkey,key,year,value])
                    i += 1

In [5]:
get_edge('bio')
get_edge('EE&IT')

### 4. Join

In [10]:
os.chdir('E:/apps/feature_ffy')

bio = pd.read_csv('lit_t1_bio.csv')
ee = pd.read_csv('lit_t1_EE&It.csv')

bc = pd.read_csv('lit_t1_c_bio.csv')
ec = pd.read_csv('lit_t1_c_EE&It.csv')

bio = bio.merge(bc,how='left',on=['gvkey','gvkey_cited','year']).fillna(0)
ee = ee.merge(ec,how='left',on=['gvkey','gvkey_cited','year']).fillna(0)

In [14]:
bc = pd.read_csv('cit_bio.csv')
ec = pd.read_csv('cit_EE&It.csv')

bio = bio.merge(bc,how='outer',on=['gvkey','gvkey_cited','year']).fillna(0)
ee = ee.merge(ec,how='outer',on=['gvkey','gvkey_cited','year']).fillna(0)

len(bio), len(ee)

(23510, 135361)

In [15]:
bp = pd.read_csv('proximity_bio.csv')
ep = pd.read_csv('proximity_EE&IT.csv')

bio = bio.merge(bp,how='left',on=['gvkey','gvkey_cited','year']).fillna(0)
ee = ee.merge(ep,how='left',on=['gvkey','gvkey_cited','year']).fillna(0)

len(bio), len(ee)

(23510, 135361)

In [16]:
bio.to_csv('X_bio_mvars.csv',index=False)
ee.to_csv('X_EE&IT_mvars.csv',index=False)